In [16]:
!pip3 install -U tensorflow

In [17]:
amazon=pd.read_csv("Eco_Friendly_Products_Test_Full.xlsx - Sheet1.csv")

In [28]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


def preprocess_text(text):
    # Remove HTML tags (if any)
    text = re.sub(r"<.*?>", "", text)
    
    # Tokenize the text into words
    tokens = nltk.word_tokenize(text)
    
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    
    # Remove punctuation
    tokens = [token for token in tokens if token.isalpha()]
    
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    # print(filtered_tokens)
    # # Lemmatization
    # lemmatizer = WordNetLemmatizer()
    # lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Join the tokens back into a single string
    # processed_text = " ".join(filtered_tokens)

    
    return filtered_tokens

In [29]:
m=amazon['Review']

In [30]:
processed_series = m.apply(preprocess_text)

In [31]:
processed_series

0                           [dries, hair, without, frizz]
1                         [glad, dries, hair, half, time]
2                                  [also, diffuser, stay]
3                             [best, dryer, ever, bought]
4       [dries, medium, length, hair, quickly, love, l...
                              ...                        
1022                                               [ugly]
1023    [works, really, good, automatic, windows, come...
1024                    [high, quality, fans, work, well]
1025                         [also, wide, gap, installed]
1026                       [fit, car, window, advertised]
Name: Review, Length: 1027, dtype: object

In [32]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.cluster import KMeans
import numpy as np

# Sample data: list of sentences
# Each sentence is represented as a list of words

# Convert each document into a TaggedDocument object
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(processed_series)]

# Train a Doc2Vec model
model = Doc2Vec(vector_size=20, window=2, min_count=1, workers=4, epochs=100)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# Generate document vectors
document_vectors = np.array([model.dv[str(i)] for i in range(len(tagged_data))])

# Perform k-Means clustering
num_clusters = 10  # Choose an optimal number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(document_vectors)

# Get cluster assignments
labels = kmeans.labels_
dict_aspects = {}
# Print out the clusters
for i in range(num_clusters):
    print(f"Cluster {i+1}:")
    for j, label in enumerate(labels):
        if label == i:
            print(f"  - {' '.join(processed_series[j])}")
            dict_aspects[i+1] = processed_series[j]


Cluster 1:
  - dries hair without frizz
  - also diffuser stay
  - dries medium length hair quickly love low setting cool shot
  - dries fast without lot heat
  - great hair dryer
  - drys hair fast long thick hair
  - get hot enough low setting
  - attractive
  - small functional
  - cleans highly recommended
  - pretty wobbly stay together well
  - bamboo straws awesome work great thick smoothies well
  - easy use easy clean
  - happy straws portable easy clean durable attractive
  - easy clean
  - perfect small trash cans even storing items want use plastic
  - buying quite years tear
  - great composter bags
  - bags hold really well break soon added wet compost bin
  - easy use
  - easy use
  - simple easy use
  - way flimsy break easily
  - like easy use effective
  - smell time like sponges
  - rinse super easily
  - durable use basic dish washing
  - easily washes top rack dishwasher
  - easily microwave break throw
  - like throw washing machine
  - heavy lovely colors
  - kid

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [33]:
dict_aspects

{1: ['utterly', 'useless', 'really', 'use', 'know'],
 2: ['high', 'quality', 'fans', 'work', 'well'],
 3: ['way',
  'bulky',
  'fit',
  'good',
  'also',
  'sensitive',
  'light',
  'hitting',
  'panel'],
 4: ['fit', 'car', 'window', 'advertised'],
 5: ['pretty',
  'easily',
  'identified',
  'seat',
  'belt',
  'might',
  'like',
  'separate',
  'repurposed',
  'vibe',
  'really',
  'well',
  'made',
  'comfortable',
  'strap',
  'soudlier',
  'flag',
  'tag',
  'nice',
  'touch',
  'neck',
  'heavy',
  'guitar',
  'like',
  'mine',
  'sg',
  'find',
  'strap',
  'want',
  'slide',
  'bit',
  'straps'],
 6: ['button',
  'activated',
  'sun',
  'angle',
  'sunshine',
  'plate',
  'driving',
  'go',
  'hassle',
  'put',
  'take',
  'often',
  'leave',
  'window',
  'hence',
  'noise',
  'issue'],
 7: ['friends',
  'new',
  'car',
  'solar',
  'panels',
  'roof',
  'thing',
  'mine',
  'costs',
  'lot',
  'less'],
 8: ['annoying',
  'reasons',
  'bags',
  'quite',
  'bulky',
  'makes',
 

In [23]:
import tensorflow_hub as hub
import numpy as np
from sklearn.cluster import KMeans
import tensorflow as tf
import tensorflow_text  # Required for text preprocessing ops in TF-Hub models. Don't remove.

# Load Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Sample sentences
sentences = m

# Generate embeddings using Universal Sentence Encoder
embeddings = embed(sentences)

# Perform k-means clustering
num_clusters = 3  # You can choose a suitable number of clusters based on your needs
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(embeddings)

# Print out the cluster assignments
labels = kmeans.labels_
print("Cluster labels:", labels)

# To display sentences that belong to each cluster
for i in range(num_clusters):
    print(f"Cluster {i+1}")
    for j, label in enumerate(labels):
        if label == i:
            print(f"  - {sentences[j]}")


ModuleNotFoundError: No module named 'tensorflow_text'

In [ ]:
!git clone  https://github.com/tensorflow/text.git

Cloning into 'text'...
remote: Enumerating objects: 9890, done.
remote: Counting objects: 100% (1523/1523), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 9890 (delta 1327), reused 1278 (delta 1231), pack-reused 8367
Receiving objects: 100% (9890/9890), 13.57 MiB | 10.67 MiB/s, done.
Resolving deltas: 100% (7223/7223), done.
